In [1]:
pip install pysal

  Using cached pysal-23.1-py3-none-any.whl (17 kB)
  Using cached giddy-2.3.4-py3-none-any.whl (61 kB)
  Using cached mgwr-2.1.2-py3-none-any.whl
  Using cached spopt-0.5.0-py3-none-any.whl (112 kB)
  Using cached spint-1.0.7-py3-none-any.whl
  Using cached access-1.1.8-py3-none-any.whl (21 kB)
  Using cached segregation-2.3.1-py3-none-any.whl (147 kB)
  Using cached libpysal-4.7.0-py3-none-any.whl (2.4 MB)
     -------------------------------------- 220.2/220.2 kB 2.7 MB/s eta 0:00:00
  Using cached tobler-0.9.0-py3-none-any.whl (22 kB)
  Using cached pointpats-2.3.0-py3-none-any.whl (59 kB)
  Using cached spglm-1.0.8-py3-none-any.whl
  Using cached esda-2.4.3-py3-none-any.whl
  Using cached splot-1.1.5.post1-py3-none-any.whl (39 kB)
  Using cached spvcm-0.3.0-py3-none-any.whl
  Using cached spaghetti-1.7.2-py3-none-any.whl (50 kB)
  Using cached quantecon-0.6.0-py3-none-any.whl (206 kB)
  Using cached rvlib-0.0.6.tar.gz (183 kB)
  Preparing metadata (setup.py): started
  Preparing me

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [103 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.10
  creating build\lib.win-amd64-3.10\rvlib
  copying rvlib\specials.py -> build\lib.win-amd64-3.10\rvlib
  copying rvlib\univariate.py -> build\lib.win-amd64-3.10\rvlib
  copying rvlib\version.py -> build\lib.win-amd64-3.10\rvlib
  copying rvlib\__init__.py -> build\lib.win-amd64-3.10\rvlib
  running egg_info
  writing rvlib.egg-info\PKG-INFO
  writing dependency_links to rvlib.egg-info\dependency_links.txt
  writing requirements to rvlib.egg-info\requires.txt
  writing top-level names to rvlib.egg-info\top_level.txt
  reading manifest file 'rvlib.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE.txt'
  writing manifest file 'rvlib.egg-info\SOURCES.txt'
  copying rvlib\metadata.yaml -> build\lib.wi

In [2]:
from pysal.lib import weights
from pysal.explore import esda
import numpy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sn
import contextily as cx

ModuleNotFoundError: No module named 'pysal'